In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
df_lending_data = pd.read_csv(file_path)

# Review the DataFrame
df_lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = df_lending_data['loan_status']

# Separate the X variable, the features
X = df_lending_data.drop(columns='loan_status')

In [9]:
# Review the y variable Series
y[:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: loan_status, dtype: int64

In [10]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [11]:
# Check the balance of our target values (0 = healthy loan, 1 = high risk of default)
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [12]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [22]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [30]:
#### Make a prediction using the training data
training_predictions = classifier.predict(X_train)
training_predictions_df = pd.DataFrame({"Prediction": training_predictions, "Actual": y_train})
training_predictions_df

,Prediction,Actual
44997,0,0
13468,0,0
52163,0,0
5330,0,0
9982,0,0
...,...,...
28028,0,0
18051,0,0
23757,0,0
6607,0,0


In [31]:
# Make a prediction using the testing data
test_predictions = classifier.predict(X_test)
test_predictions_df = pd.DataFrame({"Prediction": test_predictions, "Actual": y_test})
test_predictions_df

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [26]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
training_accuracy = accuracy_score(y_test, test_predictions)

0.9924164259182832

In [35]:
#### Print the balanced_accuracy score of the model for the training data
test_accuracy = accuracy_score(y_train, training_predictions)

0.9914878250103177

In [27]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
test_matrix = confusion_matrix(y_test, test_predictions)

array([[18679,    80],
       [   67,   558]])

In [19]:
# A helpful way of clarifying the above confusion_matrix output is by raveling labels to it:
tn, fp, fn, tp = confusion_matrix(y_test, test_predictions).ravel()
(tn, fp, fn, tp)

(18679, 80, 67, 558)

In [34]:
#### Generate a confusion matrix of the model for the training data
training_matrix = confusion_matrix(y_train, training_predictions)

array([[55980,   297],
       [  198,  1677]])

In [24]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ['Healthy Loans', 'High Risk Loans']
print(classification_report(y_test, test_predictions, target_names=target_names))

                 precision    recall  f1-score   support

  Healthy Loans       1.00      1.00      1.00     18759
High Risk Loans       0.87      0.89      0.88       625

       accuracy                           0.99     19384
      macro avg       0.94      0.94      0.94     19384
   weighted avg       0.99      0.99      0.99     19384



In [36]:
#### Print the classification report of the model for the training data
target_names = ['Healthy Loans', 'High Risk Loans']
print(classification_report(y_train, training_predictions, target_names=target_names))

                 precision    recall  f1-score   support

  Healthy Loans       1.00      0.99      1.00     56277
High Risk Loans       0.85      0.89      0.87      1875

       accuracy                           0.99     58152
      macro avg       0.92      0.94      0.93     58152
   weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Based on the testing data, the logistic regression model does a great job of predicting healthy loans, achieving 100%. However, the accuracy drops to 85% on High Risk Loans which is not only a significant drop relatively speaking, but a signficant miss rate given the importance of the topic.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [43]:
# Import the RandomOverSampler module from imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [44]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [47]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
ros_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
ros_model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
ros_predictions = ros_model.predict(X_test)
ros_predictions_df = pd.DataFrame({"Prediction": ros_predictions, "Actual": y_test})
ros_predictions_df

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [53]:
# Print the balanced_accuracy score of the model 
ros_accuracy = accuracy_score(y_test, ros_predictions)
ros_accuracy

0.9952022286421791

In [52]:
# Generate a confusion matrix for the model
ros_matrix = confusion_matrix(y_test, ros_predictions)
ros_matrix

array([[18668,    91],
       [    2,   623]])

In [51]:
# Print the classification report for the model
target_names = ['Healthy Loans', 'High Risk Loans']
print(classification_report(y_test, ros_predictions, target_names=target_names))

                 precision    recall  f1-score   support

  Healthy Loans       1.00      1.00      1.00     18759
High Risk Loans       0.87      1.00      0.93       625

       accuracy                           1.00     19384
      macro avg       0.94      1.00      0.96     19384
   weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Again, in using the oversampled data, the logistic regression model does a great job of predicting healthy loans. Achieving 100% again in this regard. 

However, the accuracy drops to 87% on High Risk Loans this time. This still represents both a significant drop relatively speaking, and a signficant miss rate given the importance of the topic.